<a href="https://colab.research.google.com/github/pkro/tensorflow_cert_training/blob/main/colab_notebooks/00_tensorflow_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# tensor fundamentals using tensorflow

- Introduction to tensors
- getting information from tensor
- manipulating tensors
- tensors and NumPy
- Using tf.function (speed up regular python functions)
- using gpus / tpus with TensorFlow
- Exercises


Introduction to tensors


In [2]:
import tensorflow as tf

print(tf.__version__)

2.9.2


In [3]:
print(tf.config.list_physical_devices('GPU'))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[]
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7710128016804890018
xla_global_id: -1
]


In [4]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [5]:
#Check number of dimentsions of a tensor (ndim = Number of DIMensions)
scalar.ndim

0

In [6]:
# create a vector
vector = tf.constant([3,4])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([3, 4], dtype=int32)>

In [7]:
# check vector dimension
vector.ndim

1

In [8]:
# matrix (more than 1 dimension)
matrix = tf.constant([[10,7],
                      [7,10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [9]:
matrix.ndim

2

It seems the dimension of the tensors is related to the number of elements in the `shape` property of the tensor.

In [10]:
# Create another matrix and specify data tyoe
another_matrix = tf.constant([[10.,7.],
                              [8., 9.],
                              [3., 4.]], dtype=tf.float16)
another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 8.,  9.],
       [ 3.,  4.]], dtype=float16)>

In [11]:
another_matrix.ndim

2

The shape seems to have the format (rows, columns, ...);

In [12]:
# note: all of the above were tensors too as e.g. a matrix is a subtype of a tensor
tensor = tf.constant([
    [[1,2,3], [4,5,6]],
    [[7,8,9], [10,11,12]],
    [[13,14,15], [16,17,18]]
])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [13]:
tensor.ndim

3

### What we learned:

- scalar: a single number
- vector: a number with direction, e.g. wind speed and direction
- matrix: 2-dimensional array of numbers
- tensor: n-dimensional array of numbers (n = any number; a 0-dimensional tensor is a scalar, a 1-dimensional tensor is a vector)


### Creating tensors with tf.Variable and tf.constant

>A variable maintains shared, persistent state manipulated by a program.

https://www.tensorflow.org/api_docs/python/tf/Variable

When in doubt, use constant and change later if needed.

In practice, creating variables / constants is taking care of by tf.

*tensors created with tf.Variable don't seem to have an ndim property?*


In [14]:
v = tf.Variable(1.)
v

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.0>

In [15]:
# does not work, shape mismatch
# v.assign([1,2])
v.assign(5.) # works
v

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=5.0>

In [16]:
changeable_tensor = tf.Variable([10,7])
unchangeable_tensor = tf.constant([4,5])
changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([4, 5], dtype=int32)>)

In [17]:
changeable_tensor[0]

<tf.Tensor: shape=(), dtype=int32, numpy=10>

In [18]:
# changeable_tensor.assign([1,2,3]) # shape mismatch
# unchangeable_tensor.assign([1,2]) # obviousle doesn't even have an assign method
changeable_tensor.assign([1,2]) # works fine

<tf.Variable 'UnreadVariable' shape=(2,) dtype=int32, numpy=array([1, 2], dtype=int32)>

In [19]:
# We can also assign single elements
changeable_tensor[0].assign(10)

<tf.Variable 'UnreadVariable' shape=(2,) dtype=int32, numpy=array([10,  2], dtype=int32)>

### Creating random tensors with tf.Variable()

Random tensors are tensors of arbitrary size containing random numbers.

They are used to initialize the middle layer of the neural network (=representation = patterns = features = weights) and then tweak them by learning.

![random initialization](https://github.com/pkro/tensorflow_cert_training/blob/main/readme_images/random_init.png?raw=1)


In [20]:
# create 2 random (but same) tensors
random_1 = tf.random.Generator.from_seed(42); # set seed for reproducibility
random_1 = random_1.normal(shape=(3,2)) # Outputs random values from a normal distribution.
random_1

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ]], dtype=float32)>

>A normal distribution is a type of continuous probability distribution in which most data points cluster toward the middle of the range, while the rest taper off symmetrically toward either extreme. The middle of the range is also known as the mean of the distribution.
[source](https://www.techtarget.com/whatis/definition/normal-distribution)

Basically a bell curve.

In [21]:
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3,2))
random_1, random_2, random_1 == random_2 # same seed, same values

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

### Shuffle the order of elements in a tensor

Example use: if a NN has a list of 15000 images, the first 10.000 of Spagghetti and the last 5000 of Ramen, it might optimize too much on the Spagghetti recognition before reaching the Ramen images. It would be better to mix the input (images) so it learns both at the same time.

In [22]:
# shuffle a tensor (valuable to shuffle data so the inherent order doesn't affect learning)
not_shuffled = tf.constant([[10,7],
                            [3,5],
                            [1,9]])
not_shuffled.ndim #2
not_shuffled


<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  5],
       [ 1,  9]], dtype=int32)>

In [23]:
# shuffles only the first dimension ("rows"), content of sub-arrays stays the same!

shuffled = tf.random.shuffle(not_shuffled)
shuffled.ndim # also 2 of course
print("random, without seed:", shuffled)

# can take a seed
shuffled = tf.random.shuffle(not_shuffled, 42)
shuffled.ndim # also 2 of course
print("STILL random even with seed:", shuffled)

random, without seed: tf.Tensor(
[[ 3  5]
 [ 1  9]
 [10  7]], shape=(3, 2), dtype=int32)
STILL random even with seed: tf.Tensor(
[[ 1  9]
 [ 3  5]
 [10  7]], shape=(3, 2), dtype=int32)


>"Operations that rely on a random seed actually derive it from two seeds: the global and operation-level seeds. This sets the global seed."

[more on the rules of how these seeds are used](https://www.tensorflow.org/api_docs/python/tf/random/set_seed)

In [24]:
tf.random.set_seed(99) # global seed
shuffled = tf.random.shuffle(not_shuffled, seed=42) # operation level seed
shuffled # stays in the same (random) order

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  5],
       [ 1,  9]], dtype=int32)>

For reproducibility of experiments, set both the global and operation level seed.

### Other ways to make tensors

### Creating tensors from NumPy arrays

Note that tensorflow has many NumPy operations (such as `ones`) already built in.

In [25]:
# creates a tensor of a given shape where all elements are 1
tf.ones(shape=(3,2), dtype=tf.int32)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 1],
       [1, 1],
       [1, 1]], dtype=int32)>

In [26]:
# create a tensor of 0s
tf.zeros(shape=(3,2), dtype=tf.int32)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[0, 0],
       [0, 0],
       [0, 0]], dtype=int32)>

In [27]:
# create a tensor filled with an arbitrary value
tf.fill([3,2], value=99) # note that shape is passed as an array instead of a tupple here

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[99, 99],
       [99, 99],
       [99, 99]], dtype=int32)>

### Turn a NumPy array into tensors

Main difference between NumPy arrays and tf tensors: tensors can be run on a GPU, otherwise very similar


In [28]:
import numpy as np

numpy_a = np.arange(1, 25, dtype=np.int32)

# Capitalization "rules":
# A = tf.constant(some_matrix) # capital for matrix or tensor
# a = tf.constant(vector) # lowercase for vector
numpy_a #  numpy array

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [29]:
# convert to tf tensor
tensor_a = tf.constant(numpy_a)
tensor_a

<tf.Tensor: shape=(24,), dtype=int32, numpy=
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)>

In [30]:
# Change shape from vector to matrix from a one-dimensional array
# the number of elements in the source array must add up to the elements required
# by the shape
A = tf.constant(numpy_a, shape=(3,8))
B = tf.constant(numpy_a)
A, A.ndim, B, B.ndim

(<tf.Tensor: shape=(3, 8), dtype=int32, numpy=
 array([[ 1,  2,  3,  4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13, 14, 15, 16],
        [17, 18, 19, 20, 21, 22, 23, 24]], dtype=int32)>,
 2,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>,
 1)

### Getting information from tensors

When dealing with tensors you need to be aware of the following attributes:

- Shape
- Rank
- Axis or dimension
- Size

![attributes](https://github.com/pkro/tensorflow_cert_training/blob/main/readme_images/tensor_attributes.png?raw=1)

In [31]:
# create rank 4 tensor
rank_4_tensor = tf.zeros([2,3,4,5]) # 2*3*4*5 "4d matrix"
rank_4_tensor, "rank: ", rank_4_tensor.ndim

(<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
 array([[[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]],
 
         [[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]],
 
         [[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]],
 
 
        [[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]],
 
         [[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]],
 
         [[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]]], dtype=float32)>, 'rank: ', 4)

In [32]:
print(rank_4_tensor[0], "\n")
print(rank_4_tensor[0][0], "\n") 
print(rank_4_tensor[0][0][0], "\n") 
print(rank_4_tensor[0][0][0][0], "\n") 

tf.Tensor(
[[[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]], shape=(3, 4, 5), dtype=float32) 

tf.Tensor(
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]], shape=(4, 5), dtype=float32) 

tf.Tensor([0. 0. 0. 0. 0.], shape=(5,), dtype=float32) 

tf.Tensor(0.0, shape=(), dtype=float32) 



In [33]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [34]:
# Get various attributes of our tensor
def tensor_info(tensor):
  print("Datatype of every element: ", tensor.dtype)
  print("Number of dimensions (rank): ", tensor.ndim)
  print("Shape: ", tensor.shape)
  print("Elements along the 0 axis: ", tensor.shape[0])
  print("Elements along the last axis: ", tensor.shape[-1])
  print("Total number of elements: ", tf.size(tensor))
  print("Total number of elements (plain): ", tf.size(tensor).numpy())

tensor_info(rank_4_tensor)

Datatype of every element:  <dtype: 'float32'>
Number of dimensions (rank):  4
Shape:  (2, 3, 4, 5)
Elements along the 0 axis:  2
Elements along the last axis:  5
Total number of elements:  tf.Tensor(120, shape=(), dtype=int32)
Total number of elements (plain):  120


### Indexing and expanding tensors

Tensors can be indexed like python lists


In [35]:
# get first 2 elements of each tensor dimension
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [36]:
# Example: remove the first element of the innermost tensors
rank_4_tensor[:, :, :, 1:]

<tf.Tensor: shape=(2, 3, 4, 4), dtype=float32, numpy=
array([[[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]],


       [[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]]], dtype=float32)>

In [37]:
# get the first element from except from the final one
rank_4_tensor[:1, :1, :1]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [38]:
# Create a rank 2 tensor
rank_2_tensor = tf.constant([[1,2], [3,4]])
tensor_info(rank_2_tensor), "\n", rank_2_tensor

Datatype of every element:  <dtype: 'int32'>
Number of dimensions (rank):  2
Shape:  (2, 2)
Elements along the 0 axis:  2
Elements along the last axis:  2
Total number of elements:  tf.Tensor(4, shape=(), dtype=int32)
Total number of elements (plain):  4


(None, '\n', <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4]], dtype=int32)>)

In [39]:
# Get last item of each row
rank_2_tensor[:, -1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([2, 4], dtype=int32)>

In [40]:
# Add extra dimension (important)

# 1st alternative
rank_3_tensor = rank_2_tensor[..., tf.newaxis] # ... = every previous axis, same as [:, :, tf.newaxis]
rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[1],
        [2]],

       [[3],
        [4]]], dtype=int32)>

In [41]:
# 2nd alternative
tf.expand_dims(rank_2_tensor, axis=-1) # -1 = expand final axis

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[1],
        [2]],

       [[3],
        [4]]], dtype=int32)>

In [42]:
tf.expand_dims(rank_2_tensor, axis=0) # expand 0-axis

<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[1, 2],
        [3, 4]]], dtype=int32)>

### Manipulating tensors (tensor operations)

**Basic  operations**

+, -, *, /

In [43]:
# add values using the additon operator
# the original tensor stays unchanged of course

tensor = tf.constant([[10,7], [3,4]])
tensor + 10 # adds 10 to all elements in the tensor


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [44]:
tensor - 1 # same for subtraction

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[9, 6],
       [2, 3]], dtype=int32)>

In [45]:
tensor * 3

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[30, 21],
       [ 9, 12]], dtype=int32)>

In [46]:
tensor / 3 # also automatically converts to float

<tf.Tensor: shape=(2, 2), dtype=float64, numpy=
array([[3.33333333, 2.33333333],
       [1.        , 1.33333333]])>

In [47]:
# using tf methods (tf.math.* and tf.* are equivalent but not all math.* methods exist on tf.*)
tf.multiply(tensor, 5), tf.math.add(tensor, 5)

(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[50, 35],
        [15, 20]], dtype=int32)>, <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[15, 12],
        [ 8,  9]], dtype=int32)>)

**For the engine to be able to split up operations, it is best to use the tf.* or tf.math.* methods, not the standard python operators**

### Matrix multiplication

In machine learning, [matrix multiplication](https://www.mathsisfun.com/algebra/matrix-multiplying.html) is the most commoon tensor operation.

A matrix multiplication is the result of the dot product (Skalarprodukt oder inneres Produkt) of rows and columns.

![dot product](https://github.com/pkro/tensorflow_cert_training/blob/main/readme_images/dot_product.png?raw=1)

[source](https://www.mathsisfun.com/algebra/matrix-multiplying.html)

[Visualize matrix multiplication!](http://matrixmultiplication.xyz/)



Rules: 

1) The *inner* dimensions must match, meaning that **the number of columns of the 1st matrix must equal the number of rows of the 2nd matrix.**

2) The result will have the same number of rows as the 1st matrix and the same number of columns as the 2nd matrix (size is the same as the *outside* numbers)

Inner / outer refers to the position of the dimensions in the multiplication:

- **Inner**: Rows x **COLUMNS** * **ROWS** x Columns
- **Outer**: **ROWS** x Columns * Rows x **COLUMNS**

![matrix multiplication](https://github.com/pkro/tensorflow_cert_training/blob/main/readme_images/matrix_mult.png?raw=1)


In [48]:
# Matrix multiplication in tensorflow

tf.matmul(tensor, tensor) # same as tf.linalg.matmul, used from here on out

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [49]:

# "reak" Matrix multiplication with python operator "@"
tensor @ tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [50]:
# Element-wise matrix multiplication with python operator "*"
tensor * tensor # (10*10, 7*7, 3*3, 4*4)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  49],
       [  9,  16]], dtype=int32)>

In [52]:
# Create 2 tensors of shape (3,2)
X = tf.constant([[1,2], [3,4], [5,6]]) # 3 rows, 2 columns
Y = tf.constant([[7,8], [9,10], [11,12]]) # 3 rows, 2 columns

tf.matmul(X, Y) # matrix size incompatible! See rule #1, inner dimensions must match - num rows m1 must be num cols m2

InvalidArgumentError: ignored

In [53]:
# Let's change the shape of Y (doesn't the matrix have an entire different meaning then?)
print(Y, "\n")
tf.reshape(Y, shape=(2,3))

tf.Tensor(
[[ 7  8]
 [ 9 10]
 [11 12]], shape=(3, 2), dtype=int32) 



<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 7,  8,  9],
       [10, 11, 12]], dtype=int32)>

In [54]:
# try to multipley X by reshaped Y
X @ tf.reshape(X, shape=(2, 3))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 9, 12, 15],
       [19, 26, 33],
       [29, 40, 51]], dtype=int32)>

In [55]:
X.shape, tf.reshape(Y, shape=(2,3)).shape

(TensorShape([3, 2]), TensorShape([2, 3]))

In [56]:
tf.matmul(X, tf.reshape(Y, shape=(2,3))) # works the same as @

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [57]:
# try changing X instead of Y
tf.reshape(X, shape=(2,3)) @ Y

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 58,  64],
       [139, 154]], dtype=int32)>

-> Reshaping X works, too, but the result is different as now the *outer* dimensions is 2 for both.

In [58]:
# we can do the same with transpose, but it does a different thing than reshape
X, tf.transpose(X), tf.reshape(X, shape=(2,3))

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>, <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 3, 5],
        [2, 4, 6]], dtype=int32)>, <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 5, 6]], dtype=int32)>)

- [transpose](https://www.tensorflow.org/api_docs/python/tf/transpose) flips the axes' 
- [reshape](https://www.tensorflow.org/api_docs/python/tf/reshape) assigns the numbers lineary from top left to bottom right in the same order but in a different shape ("Given tensor, this operation returns a new tf.Tensor that has the same values as tensor in the same order, except with a new shape given by shape.")

They don't give the same results in the resulting matrix or the multiplication result.

In [59]:
# try multiplication with transpose rather than reshape
tf.transpose(X) @ Y

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

**

**The dot product**

Matrix multiplication is also referred to as the dot product.

You can perform matrix multiplication using:

- `tf.matmul()`
- [`tf.tensordot()`](https://www.tensorflow.org/api_docs/python/tf/tensordot)


In [60]:
X, Y # little reminder

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>, <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

In [61]:
# Perfomr dot product on X and Y (requires X or Y to be transposed)
tf.tensordot(tf.transpose(X), Y, axes=1)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

In [62]:
# Perform matrix multiplication between X and Y (transposed)
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

In [63]:
# Perform matrix multiplication between X and Y (reshaped)
tf.matmul(X, tf.reshape(Y, shape=(2,3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [64]:
# Check the values of Y, reshape Y and transpose Y
print("Normal Y: ")
print(Y, "\n")

print("Y reshaped to (2, 3): ")
print(tf.reshape(Y, (2, 3)), "\n")

print("Y transposed: ")
print(tf.transpose(Y), "\n")

Normal Y: 
tf.Tensor(
[[ 7  8]
 [ 9 10]
 [11 12]], shape=(3, 2), dtype=int32) 

Y reshaped to (2, 3): 
tf.Tensor(
[[ 7  8  9]
 [10 11 12]], shape=(2, 3), dtype=int32) 

Y transposed: 
tf.Tensor(
[[ 7  9 11]
 [ 8 10 12]], shape=(2, 3), dtype=int32) 



In [65]:
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

Most matrix modification is done behind the scenes in tensorflow, meaning it doesn't have to be done by hand a lot.

**Generally, if tensor dimensions don't line up, one of the tensors should be transposed and not reshaped to satisfy the matrix multiplication rules.**

### Changing the datatype of tensors

Default datatype is mostly int32 or float32, depending on the data inside the tensor. Models / data with lower precision types (or mixed precision) can run faster on gpus and in general.

https://www.tensorflow.org/guide/mixed_precision


In [80]:
# create tensor with default datatype (float32 fpr floats)
B = tf.constant([1.7, 7.4])
B.dtype

tf.float32

In [69]:
# int32 for integers
C = tf.constant([7,10])
C.dtype

tf.int32

In [73]:
# change from float32 to float16 (reduced precission)
D = tf.cast(C, tf.float16)
D, D.dtype

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([ 7., 10.], dtype=float16)>,
 tf.float16)

In [75]:
# change from int32 to float32
E = tf.cast(C, dtype=tf.float32)
E

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 7., 10.], dtype=float32)>

In [81]:
# casting from float to int removes decimals
F = tf.cast(B, dtype=tf.int32)
F

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 7], dtype=int32)>

### Aggregating tensors
